# Kernels 

各种核函数汇总

### 线性核函数

$$
K(x,y) = x \times y + c
$$

In [1]:
def linear_kernel(X, Y, c0):
    """
    Parameters
    ----------
    c0 : float
        An "inhomogeneity" parameter. When `c0` = 0, the kernel is said to be
        homogenous. Default is 1.
    """
    return X @ Y.T + c0

### 多项式核函数 

$$
K(x,y) = (\gamma(x \times y) + c)^d
$$

In [2]:
def polynomial_kernel(X, Y, d, gamma, c0):
    """
    Parameters
    ----------
    d : int
        Degree of the polynomial kernel. Default is 3.
    gamma : float or None
        A scaling parameter for the dot product between `x` and `y`,
        determining the amount of smoothing/resonlution of the kernel.
        Larger values result in greater smoothing. If None, defaults to 1 /
        `C`.  Sometimes referred to as the kernel bandwidth.  Default is
        None.
    c0 : float
        Parameter trading off the influence of higher-order versus lower-order
        terms in the polynomial. If `c0` = 0, the kernel is said to be
        homogenous. Default is 1.
    """
    gamma = 1 / X.shape[1] if gamma is None else gamma
    return (gamma * (X @ Y.T) + c0) ** d

### Radial basis 核函数 

$$
K(x,y) = e^{-\gamma\|x -y\|^2}
$$

pairwise_l2_distances:

A fast, vectorized way to compute pairwise l2 distances between rows in `X` and `Y`.

$$
\begin{align*}
        d[i, j]  &=  \sqrt{(x_i - y_i) @ (x_i - y_i)} \\
                 &=  \sqrt{sum (x_i - y_j)^2} \\
                 &=  \sqrt{sum (x_i)^2 - 2 x_i y_j + (y_j)^2} \\
\end{align*}
$$

In [3]:
def pairwise_l2_distances(X, Y):
    """
    Parameters
    ----------
    X : :py:class:`ndarray <numpy.ndarray>` of shape `(N, C)`
        Collection of `N` input vectors
    Y : :py:class:`ndarray <numpy.ndarray>` of shape `(M, C)`
        Collection of `M` input vectors. If None, assume `Y` = `X`. Default is
        None.

    Returns
    -------
    dists : :py:class:`ndarray <numpy.ndarray>` of shape `(N, M)`
        Pairwise distance matrix. Entry (i, j) contains the `L2` distance between
        :math:`x_i` and :math:`y_j`.
    """
    D = -2 * X @ Y.T + np.sum(Y ** 2, axis=1) + np.sum(X ** 2, axis=1)[:, np.newaxis]
    D[D < 0] = 0  # clip any value less than 0 (a result of numerical imprecision)
    return np.sqrt(D)

def RBF_kernel(X, Y, sigma):
    """
    Parameters
    ----------
    sigma : float or array of shape `(C,)` or None
        A scaling parameter for the vectors **x** and **y**, producing an
        isotropic kernel if a float, or an anistropic kernel if an array of
        length `C`.  Larger values result in higher resolution / greater
        smoothing. If None, defaults to :math:`\sqrt(C / 2)`. Sometimes
        referred to as the kernel 'bandwidth'. Default is None.
    """
    sigma = np.sqrt(X.shape[1] / 2) if sigma is None else sigma
    return np.exp(-0.5 * pairwise_l2_distances(X / sigma, Y / sigma) ** 2)